In [2]:
from fastapi import FastAPI, Request, Response
from pydantic import BaseModel
import httpx
import asyncio

app = FastAPI()

In [12]:
async def buildBotMessage(chatId, content):
    """
    채팅에 봇 메시지를 만드는 함수
    :param chatId: 채팅방 UUID
    :param content: 콘텐츠
    :return:
    """
    async with httpx.AsyncClient() as client:
        try:
            url = f"https://localhost:8080/agent/chat/{chatId}"

            response = await client.post(url, json={"content": content})

            if response.status_code != 200:
                print("Build Bot Message Fail")

            body = response.json()
            return body.get("messageId")
        except Exception as e:
            print("Build Bot Message Fail",e)
            return None

In [ ]:
async def updateBotMessage(messageId, content):
    """
    봇이 보낸 채팅 메시지를 업데이트하는 함수
    :param messageId: 메시지 id
    :param content: 업데이트 된 메시지
    :return:
    """
    async with httpx.AsyncClient() as client:
        try:
            url = f"https://localhost:8080/agent/chat/edit/{messageId}"

            response = await client.post(url, json={"content": content})

            if response.status_code != 200:
                print("Build Bot Message Fail")

            return True
        except Exception as e:
            print("Build Bot Message Fail",e)
            return False

In [4]:
class RequestPayload(BaseModel):
    chatId: str
    content: str

@app.post("/request")
async def handle_request(payload: RequestPayload):
    messageId = await buildBotMessage(chatId=payload.chatid, content=payload.content)
    asyncio.run(updateBotMessage(messageId, payload.content))

    return Response(status_code=200)